<div style="text-align: right" align="right"><i>Peter Norvig, December 2025</i></div>

# Advent of Code 2025: The AI LLM Edition

*I  enjoy doing the [**Advent of Code**](https://adventofcode.com/) (AoC) programming puzzles, and **my** solutions are [**over here**](Advent2025.ipynb).*

*In **this** notebook I show some solutions by various AI Large Language Models: Gemini, Claude, and ChatGPT. In order to tell the difference between what the LLM generates and what I say, my comments (like this one) are in italics, and my prompts given to the AI LLMs are in **bold italics**. The LLM responses are in* regular roman font.

*In order to really understand what's going on here, you'll have to look at the problem descriptions at [**Advent of Code**](https://adventofcode.com/2025) for each [**Day**](https://adventofcode.com/2025/day/1).*

*Each day I'll choose a model and give it the prompt* "***Write code to solve the following problem:***" *followed by the full text of the AoC Part 1 problem description. Then I'll prompt again with* "***There is a change to the specification:***" *followed by the AoC Part 2 description.*

*For brevity, I have removed some of the LLM output, such as:*
- *Prose analysis of the problem and descriptions of the programs. (In most cases these were accurate and thorough!)*
- *The "`#!/usr/bin/env python3`" at the top of the file*
- *The "`__main__`" idiom for running code from the command line*
- *Usage examples on how to run the program on test input*



# Day 0



*On Day 0 I load my [**AdventUtils.ipynb**](AdventUtils.ipynb) and set the`current_year` so I can access my input files with `get_text(day_number)` and can use my `answer` function to verify whether the AI systems get the right answer.*

In [1]:
%run AdventUtils.ipynb
current_year = 2025

# [Day 1](https://adventofcode.com/2025/day/1) Gemini 3 Pro

*For **Part 1** I started with the Gemini 3 Pro Fast model, which produced this following code:*

In [2]:
def solve_safe_password(rotations):
    """
    Simulates the safe dial rotations and counts how many times the dial
    lands on 0.

    Args:
        rotations: A list of strings, each representing a rotation (e.g., "L68").

    Returns:
        The total number of times the dial points at 0 after a rotation.
    """
    # The dial starts at 50
    current_position = 50
    zero_count = 0
    
    # The dial has numbers 0 through 99, so all math is modulo 100
    MODULO = 100

    for rotation in rotations:
        # 1. Parse the instruction
        direction = rotation[0]
        # Distance is the rest of the string converted to an integer
        distance = int(rotation[1:])

        # 2. Determine the rotation value
        if direction == 'R':
            rotation_value = distance
        elif direction == 'L':
            # Left rotations subtract from the current position
            rotation_value = -distance
        else:
            # Should not happen based on problem description
            print(f"Error: Unknown direction '{direction}' in rotation: {rotation}")
            continue

        # 3. Calculate the new position and apply Modulo 100
        # The standard Python % operator handles negative numbers correctly
        # for a positive modulus (e.g., (-1 - 99) % 100 == 0).
        new_position = (current_position + rotation_value) % MODULO

        # 4. Check for Zero
        if new_position == 0:
            zero_count += 1
        
        # 5. Update Position
        current_position = new_position
        
        # Optional: Print step-by-step for debugging/verification
        # print(f"Rotation {rotation}: new position = {new_position}, zero_count = {zero_count}")

    return zero_count

*A bit verbose and over-commented, but pretty nice. Below I fetch the Day 1 data with `get_text(1)`, split it into lines (as expected by Gemini's `solve_safe_password` function),  and verify that the answer is correct:*

In [3]:
rotations = get_text(1).splitlines()

answer(puzzle=1.1, solution=1182, code=lambda: 
       solve_safe_password(rotations))

Puzzle  1.1:   .0012 seconds, answer 1182            correct

*For **Part 2** I gave Gemini Fast the updated instructions, and again it had a nice analysis and the following code:*

In [4]:
import math

def solve_new_safe_password(rotations):
    """
    Simulates the safe dial rotations and counts every time the dial
    points at 0, including during the rotation movement.

    Args:
        rotations: A list of strings, each representing a rotation (e.g., "L68").

    Returns:
        The total number of times the dial points at 0.
    """
    # The dial starts at 50
    current_position = 50
    total_zero_crossings = 0
    MODULO = 100

    for rotation in rotations:
        direction = rotation[0]
        distance = int(rotation[1:])
        
        S = current_position # Start Position
        D = distance         # Distance
        
        crossings_for_this_rotation = 0

        if direction == 'L':
            # --- LEFT ROTATION ---
            # Dial moves from S -> S-1 -> ... -> 0 -> 99 -> ...
            
            # 1. Distance from S to the first 0: S clicks
            # If D is large enough to cover this distance, the first 0 is crossed.
            
            if D >= S:
                # The first crossing (at position 0) is achieved
                crossings_for_this_rotation += 1
                
                # 2. Remaining distance after first 0 crossing
                remaining_distance = D - S
                
                # 3. Subsequent crossings: The dial crosses 0 every 100 clicks
                # after the first one.
                crossings_for_this_rotation += remaining_distance // MODULO
                
        elif direction == 'R':
            # --- RIGHT ROTATION ---
            # Dial moves from S -> S+1 -> ... -> 99 -> 0 -> 1 -> ...
            
            # 1. Distance from S to the first 0: 100 - S clicks
            # (e.g., 50 to 0 is 50 clicks; 99 to 0 is 1 click)
            distance_to_first_zero = MODULO - S
            
            if D >= distance_to_first_zero:
                # The first crossing (at position 0) is achieved
                crossings_for_this_rotation += 1
                
                # 2. Remaining distance after first 0 crossing
                remaining_distance = D - distance_to_first_zero
                
                # 3. Subsequent crossings: The dial crosses 0 every 100 clicks
                crossings_for_this_rotation += remaining_distance // MODULO
        
        # Add the crossings for this rotation to the total
        total_zero_crossings += crossings_for_this_rotation

        # Calculate the final position to set up for the next rotation
        if direction == 'R':
            rotation_value = distance
        else: # 'L'
            rotation_value = -distance
            
        current_position = (current_position + rotation_value) % MODULO

    return total_zero_crossings

*Note that the code does `import math` but then never uses the math module. (I've done that, but I try to eventually clean things up. Maybe that's the different between Gemini "Fast" and Gemini "Thinking" models.)*

*Here I check whether the code is correct:*

In [5]:
answer(1.2, 6907, lambda: 
       solve_new_safe_password(rotations))

Puzzle  1.2:   .0022 seconds, answer 7509            WRONG; EXPECTED ANSWER IS 6907

*Too bad; the code has a **bug**. I switched to the Gemini 3 Pro "Thinking" model and gave the prompt "**That's not quite right.**" It responded with*

It seems I made a subtle logic error in my previous calculation for the Right (R) rotation.
When the dial is at 0, the distance to the *next* zero is **100**, not 0.

*which is exactly the error I made on *my* first attempt! Gemini produced the following updated code:*

In [6]:
def solve_safe_password_method_2(rotations):
    """
    Simulates the safe dial rotations and counts every time the dial
    points at 0 (Method 0x434C49434B).
    
    Correctly handles cases where the dial starts at 0.
    """
    current_position = 50
    total_zero_crossings = 0
    MODULO = 100

    for rotation in rotations:
        direction = rotation[0]
        distance = int(rotation[1:])
        
        # Determine distance required to hit the *next* 0
        if direction == 'L':
            if current_position == 0:
                dist_to_first_zero = 100
            else:
                dist_to_first_zero = current_position
                
        elif direction == 'R':
            if current_position == 0:
                dist_to_first_zero = 100
            else:
                dist_to_first_zero = 100 - current_position

        # Check if the rotation distance is enough to reach the first zero
        if distance >= dist_to_first_zero:
            # We hit 0 at least once
            count = 1
            
            # Remove the distance traveled to the first zero
            remaining_distance = distance - dist_to_first_zero
            
            # Add any subsequent full rotations (every 100 clicks hits 0 again)
            count += remaining_distance // 100
            
            total_zero_crossings += count

        # Update position for next step
        if direction == 'R':
            current_position = (current_position + distance) % 100
        else:
            current_position = (current_position - distance) % 100

    return total_zero_crossings

*This second try is indeed correct:*

In [7]:
answer(1.2, 6907, lambda: 
       solve_safe_password_method_2(rotations))

Puzzle  1.2:   .0017 seconds, answer 6907            correct

# [Day 2](https://adventofcode.com/2025/day/2) Claude Opus 4.5

*I gave Claude the instructions for **Part 1** and it wrote some code that produces the correct answer but prints a lot of unneccessary debugging output along the way. I prompted it to "**Change the code to not print anything, just return the answer**" and got the following:*

In [8]:
def find_invalid_ids_in_range(start, end):
    invalid_ids = []
    min_digits = len(str(start))
    max_digits = len(str(end))
    
    for total_digits in range(min_digits, max_digits + 1):
        if total_digits % 2 != 0:
            continue
        
        half_digits = total_digits // 2
        pattern_start = 1 if half_digits == 1 else 10 ** (half_digits - 1)
        pattern_end = 10 ** half_digits - 1
        for pattern in range(pattern_start, pattern_end + 1):
            pattern_str = str(pattern)
            invalid_id = int(pattern_str + pattern_str)
            if start <= invalid_id <= end:
                invalid_ids.append(invalid_id)
    
    return invalid_ids


def solve(input_str):
    input_str = input_str.strip().replace('\n', '').replace(' ', '').rstrip(',')
    total = 0
    for r in input_str.split(','):
        if r:
            start, end = map(int, r.split('-'))
            total += sum(find_invalid_ids_in_range(start, end))
    return total

*This code is overall rather nice, but conspicously lacks comments and doc strings. It uses the more efficient "enumerate over the first half of the digit string" strategy, but is not precise in narrowing down the range it enumerates over. For example, for the range "999000-109000", this code will enumerate the range (100, 1000), when it could enumerate just the range (999, 1000).*

*I verified that the code gives the correct answer:*

In [9]:
id_ranges = get_text(2)

answer(2.1, 23560874270, lambda:
       solve(id_ranges))

Puzzle  2.1:   .1217 seconds, answer 23560874270     correct

*When given the **Part 2** instructions, Claude wrote the following code:*

In [10]:
def find_invalid_ids_in_range(start, end):
    invalid_ids = set()
    min_digits = len(str(start))
    max_digits = len(str(end))
    
    for total_digits in range(min_digits, max_digits + 1):
        # Pattern length can be 1 to total_digits//2 (need at least 2 repetitions)
        for pattern_len in range(1, total_digits // 2 + 1):
            if total_digits % pattern_len != 0:
                continue
            
            repetitions = total_digits // pattern_len
            if repetitions < 2:
                continue
            
            pattern_start = 1 if pattern_len == 1 else 10 ** (pattern_len - 1)
            pattern_end = 10 ** pattern_len - 1
            
            for pattern in range(pattern_start, pattern_end + 1):
                pattern_str = str(pattern)
                invalid_id = int(pattern_str * repetitions)
                if start <= invalid_id <= end:
                    invalid_ids.add(invalid_id)
    
    return invalid_ids


def solve(input_str):
    input_str = input_str.strip().replace('\n', '').replace(' ', '').rstrip(',')
    total = 0
    for r in input_str.split(','):
        if r:
            start, end = map(int, r.split('-'))
            total += sum(find_invalid_ids_in_range(start, end))
    return total

*Claude's code produces the correct answer:*

In [11]:
answer(2.2, 44143124633, lambda:
       solve(id_ranges))

Puzzle  2.2:   .1282 seconds, answer 44143124633     correct

# [Day 3](https://adventofcode.com/2025/day/3) ChatGPT 5.1 Auto

*For **Part 1** ChatGPT gave a very brief analysis of the problem and then wrote a program that was designed to be called from the command line, using the `"__main__"` idiom. I told it "**I don't want to run it like that, I want a function that I can pass the input text and have it return an int**" and it produced this code (conspicuously lacking comments or doc strings):*

In [12]:
def total_output_joltage(input_text: str) -> int:
    def max_joltage_for_bank(bank: str) -> int:
        max_first = -1
        max_pair = 0
        for ch in bank:
            if not ch.isdigit():
                continue
            d = int(ch)
            if max_first != -1:
                pair = 10 * max_first + d
                if pair > max_pair:
                    max_pair = pair
            if d > max_first:
                max_first = d
        return max_pair

    total = 0
    for line in input_text.strip().splitlines():
        line = line.strip()
        if line:
            total += max_joltage_for_bank(line)
    return total

I verified this is correct:

In [13]:
banks = get_text(3)

answer(3.1, 17085, lambda:
       total_output_joltage(banks))

Puzzle  3.1:   .0038 seconds, answer 17085           correct

*For **Part 2** ChatGPT did well (and for some reason included comments and doc strings this time):*

In [14]:
def total_output_joltage(input_text: str, k: int = 12) -> int:
    """
    Given the full puzzle input as a string, return the total maximum joltage
    when, for each line (bank), you must choose exactly k digits in order to
    form the largest possible number.
    """

    def max_joltage_for_bank(bank: str, k: int) -> int:
        # Extract digits from the line
        digits = [int(ch) for ch in bank if ch.isdigit()]
        n = len(digits)
        if k > n:
            raise ValueError(f"Bank has only {n} digits, cannot pick {k}.")

        # We will remove exactly n - k digits to maximize the resulting sequence
        removals = n - k
        stack = []

        for d in digits:
            # While we can still remove digits, and the last digit in the stack
            # is smaller than the current digit, pop it to make the number larger.
            while stack and removals > 0 and stack[-1] < d:
                stack.pop()
                removals -= 1
            stack.append(d)

        # If we still have removals left, remove from the end
        if removals > 0:
            stack = stack[:-removals]

        # Now stack should have length n - original_removals == k
        chosen = stack[:k]
        return int("".join(map(str, chosen)))

    total = 0
    for line in input_text.strip().splitlines():
        line = line.strip()
        if not line:
            continue
        total += max_joltage_for_bank(line, k)

    return total

In [15]:
answer(3.2, 169408143086082, lambda:
       total_output_joltage(banks))

Puzzle  3.2:   .0069 seconds, answer 169408143086082 correct

# [Day 4](https://adventofcode.com/2025/day/4): Gemini 3 Pro

*Gemini produced a solution to **Part 1** that is straightforward and efficient, although perhaps could use some abstraction (e.g. if they had a function to count neighbors, they wouldn't need the `continue`).*

In [16]:
def count_accessible_rolls(grid_str):
    """
    Parses a grid string and counts paper rolls (@) that have
    fewer than 4 neighbors of the same type.
    """
    # Parse input into a list of lists (2D grid)
    # .strip() removes leading/trailing whitespace to handle clean input
    lines = grid_str.strip().split('\n')
    grid = [list(line) for line in lines]
    
    rows = len(grid)
    cols = len(grid[0])
    
    accessible_count = 0
    
    # Directions for 8 neighbors: (row_offset, col_offset)
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    for r in range(rows):
        for c in range(cols):
            # We only care if the current spot is a paper roll
            if grid[r][c] != '@':
                continue
                
            neighbor_rolls = 0
            
            # Check all 8 neighbors
            for dr, dc in directions:
                nr, nc = r + dr, c + dc
                
                # Check boundaries
                if 0 <= nr < rows and 0 <= nc < cols:
                    if grid[nr][nc] == '@':
                        neighbor_rolls += 1
            
            # The Condition: fewer than 4 adjacent rolls
            if neighbor_rolls < 4:
                accessible_count += 1
                
    return accessible_count

Here I verify that the code does produce the correct answer:

In [17]:
grid = get_text(4)

answer(4.1, 1569, lambda:
       count_accessible_rolls(grid))

Puzzle  4.1:   .0170 seconds, answer 1569            correct

*For **Part 2**, Gemini chose the "repeatedly remove acessible rolls" approach rather than the more efficient (and arguable more elegant) "remove a roll and check its neighbors" approach.*

In [18]:
def count_total_removable_rolls(grid_str):
    """
    Parses a grid string and iteratively removes paper rolls (@) 
    that have fewer than 4 neighbors of the same type.
    Returns the total number of rolls removed.
    """
    # Parse input into a list of lists (2D grid)
    lines = grid_str.strip().split('\n')
    grid = [list(line) for line in lines]
    
    rows = len(grid)
    cols = len(grid[0])
    
    total_removed_count = 0
    
    # Directions for 8 neighbors
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    while True:
        rolls_to_remove = []
        
        # 1. Identify all rolls that can be removed in this pass
        for r in range(rows):
            for c in range(cols):
                if grid[r][c] != '@':
                    continue
                    
                neighbor_rolls = 0
                for dr, dc in directions:
                    nr, nc = r + dr, c + dc
                    if 0 <= nr < rows and 0 <= nc < cols:
                        if grid[nr][nc] == '@':
                            neighbor_rolls += 1
                
                # Condition: fewer than 4 adjacent rolls
                if neighbor_rolls < 4:
                    rolls_to_remove.append((r, c))
        
        # 2. If nothing to remove, we are done
        if not rolls_to_remove:
            break
            
        # 3. Remove them (update grid and count)
        total_removed_count += len(rolls_to_remove)
        for r, c in rolls_to_remove:
            grid[r][c] = '.'  # Mark as empty space
            
    return total_removed_count

In [19]:
answer(4.2, 9280, lambda:
       count_total_removable_rolls(grid))

Puzzle  4.2:   .3880 seconds, answer 9280            correct

*I asked Gemini "Can you do it more efficiently" and it replied*

The previous implementation scanned the entire grid repeatedly (`O(Rows * Cols * Iterations)`). I've replaced it with a Worklist/Queue algorithm that checks neighbors only when necessary. This reduces the complexity to be roughly proportional to the number of rolls, which is significantly faster for large grids.

*This is the same approach that I used in my solution.*

In [20]:
def count_total_removable_rolls(grid_str):
    """
    Parses a grid string and iteratively removes paper rolls (@) 
    that have fewer than 4 neighbors of the same type.
    Returns the total number of rolls removed.
    
    Optimized to use a worklist (queue) approach instead of full grid scans.
    """
    # Parse input into a list of lists (2D grid)
    lines = grid_str.strip().split('\n')
    grid = [list(line) for line in lines]
    
    rows = len(grid)
    cols = len(grid[0])
    
    total_removed_count = 0
    
    # Directions for 8 neighbors
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1),  (1, 0),  (1, 1)
    ]
    
    def count_neighbors(r, c):
        count = 0
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                if grid[nr][nc] == '@':
                    count += 1
        return count

    # 1. Initial scan: Find all rolls that are immediately removable
    queue = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '@':
                if count_neighbors(r, c) < 4:
                    queue.append((r, c))
    
    # 2. Process the queue
    # Using a list as a queue. Duplicates are handled by checking grid state.
    head = 0
    while head < len(queue):
        r, c = queue[head]
        head += 1
        
        # If already removed (e.g., added to queue multiple times), skip
        if grid[r][c] != '@':
            continue
            
        # Remove the roll
        grid[r][c] = '.'
        total_removed_count += 1
        
        # Check its neighbors
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                # If neighbor is a roll, check if it became removable
                if grid[nr][nc] == '@':
                    if count_neighbors(nr, nc) < 4:
                        queue.append((nr, nc))
            
    return total_removed_count

In [21]:
answer(4.2, 9280, lambda:
       count_total_removable_rolls(grid))

Puzzle  4.2:   .0874 seconds, answer 9280            correct

# [Day 5](https://adventofcode.com/2025/day/5): Claude Opus 4.5

*Claude produces a straightforward program that solves **Part 1** just fine and demonstrates good use of abstraction. This time it has nice doc strings; for Day 2 it had none. Go figure.*

In [22]:
"""
Solution for the Fresh Ingredients puzzle.
Parses a database with fresh ingredient ID ranges and available ingredient IDs,
then counts how many available ingredients are fresh.
"""

def parse_input(input_text):
    """Parse the input into ranges and ingredient IDs."""
    parts = input_text.strip().split('\n\n')
    
    # Parse fresh ranges
    ranges = []
    for line in parts[0].strip().split('\n'):
        start, end = map(int, line.split('-'))
        ranges.append((start, end))
    
    # Parse available ingredient IDs
    ingredient_ids = []
    for line in parts[1].strip().split('\n'):
        ingredient_ids.append(int(line))
    
    return ranges, ingredient_ids


def is_fresh(ingredient_id, ranges):
    """Check if an ingredient ID falls within any fresh range."""
    for start, end in ranges:
        if start <= ingredient_id <= end:
            return True
    return False


def count_fresh_ingredients(ranges, ingredient_ids):
    """Count how many ingredient IDs are fresh."""
    count = 0
    for ingredient_id in ingredient_ids:
        if is_fresh(ingredient_id, ranges):
            count += 1
    return count


def solve(input_text):
    """Solve the puzzle and return the count of fresh ingredients."""
    ranges, ingredient_ids = parse_input(input_text)
    return count_fresh_ingredients(ranges, ingredient_ids)

*Here I feed the input to `solve` and verify that the answer is correct:*

In [23]:
input_text = get_text(5)

answer(5.1, 635, lambda:
       solve(input_text))

Puzzle  5.1:   .0053 seconds, answer 635             correct

*For **Part 2**, Claude chose to sort ranges, then merge them, then count the total of the sizes of the ranges. That's a reasonable approach–it involves an extra step of merging ranges (instead of just counting on the fly) but it could be useful to have the merged ranges.*

In [24]:
"""
Solution for the Fresh Ingredients puzzle - Part 2.
Count all unique ingredient IDs that are considered fresh by any range.
"""

def parse_input(input_text):
    """Parse the input to get fresh ranges (ignore ingredient IDs section)."""
    parts = input_text.strip().split('\n\n')
    
    # Parse fresh ranges (only first section matters now)
    ranges = []
    for line in parts[0].strip().split('\n'):
        start, end = map(int, line.split('-'))
        ranges.append((start, end))
    
    return ranges


def merge_ranges(ranges):
    """Merge overlapping ranges to count unique IDs efficiently."""
    if not ranges:
        return []
    
    # Sort ranges by start value
    sorted_ranges = sorted(ranges)
    
    merged = [sorted_ranges[0]]
    
    for current_start, current_end in sorted_ranges[1:]:
        last_start, last_end = merged[-1]
        
        # Check if current range overlaps or is adjacent to the last merged range
        if current_start <= last_end + 1:
            # Merge by extending the end if needed
            merged[-1] = (last_start, max(last_end, current_end))
        else:
            # No overlap, add as new range
            merged.append((current_start, current_end))
    
    return merged


def count_fresh_ids(ranges):
    """Count total unique fresh ingredient IDs."""
    merged = merge_ranges(ranges)
    
    total = 0
    for start, end in merged:
        # Count is inclusive: end - start + 1
        total += end - start + 1
    
    return total


def solve(input_text):
    """Solve part 2 and return the count of all fresh ingredient IDs."""
    ranges = parse_input(input_text)
    return count_fresh_ids(ranges)

In [25]:
answer(5.2, 369761800782619, lambda:
       solve(input_text))

Puzzle  5.2:   .0002 seconds, answer 369761800782619 correct

# [Day 6](https://adventofcode.com/2025/day/6): ChatGPT 5.1 Auto

*For **Part 1** ChatGPT produced a program that is correct, but has poor abstraction, with one long 63-line function. (It also contains a pet peeve of mine: in lines 17–20 the pattern "`if some_boolean: True else: False`" can always be replaced with "`some_boolean`".)*

In [26]:
import re
from functools import reduce
from operator import add, mul

def solve(lines):
    if not lines:
        return 0

    # Preserve internal spaces; pad all lines to same width
    width = max(len(line) for line in lines)
    grid = [line.rstrip('\n').ljust(width) for line in lines]
    h = len(grid)

    # Identify separator columns: columns that are all spaces
    sep = []
    for c in range(width):
        if all(grid[r][c] == ' ' for r in range(h)):
            sep.append(True)
        else:
            sep.append(False)

    # Find ranges of columns that form individual problems
    problem_ranges = []
    in_problem = False
    start = 0
    for c in range(width):
        if not sep[c]:
            if not in_problem:
                in_problem = True
                start = c
        else:
            if in_problem:
                in_problem = False
                problem_ranges.append((start, c - 1))
    if in_problem:
        problem_ranges.append((start, width - 1))

    grand_total = 0

    # Process each problem
    for c_start, c_end in problem_ranges:
        numbers = []

        # All rows except the last contain numbers (potentially)
        for r in range(h - 1):
            segment = grid[r][c_start:c_end + 1]
            for match in re.finditer(r'\d+', segment):
                numbers.append(int(match.group(0)))

        # Last row contains the operator (+ or *)
        bottom_segment = grid[h - 1][c_start:c_end + 1]
        if '+' in bottom_segment:
            op = add
        elif '*' in bottom_segment:
            op = mul
        else:
            # No valid operator found; skip this malformed problem
            continue

        if not numbers:
            continue

        # Fold numbers using the chosen operator
        result = reduce(op, numbers)
        grand_total += result

    return grand_total

*Here I interface my input data to ChatGPT's function and verify it is correct:*

In [27]:
lines = get_text(6).splitlines()

answer(6.1, 5877594983578, lambda:
       solve(lines))

Puzzle  6.1:   .0079 seconds, answer 5877594983578   correct

*For **Part 2** we get an even-longer function, with even more complications. For example in "Determine the operator"; the 14 lines could be replaced with `op = (add if '+' in segment else mul)`. I think ChatGPT ws trying to be robust against errors in the input.*

In [28]:
import re
from functools import reduce
from operator import add, mul

def solve(lines):
    if not lines:
        return 0

    # Normalize lines to a rectangle, preserving internal spaces
    width = max(len(line) for line in lines)
    grid = [line.rstrip('\n').ljust(width) for line in lines]
    h = len(grid)

    # A separator column is all spaces -> separates problems
    sep = []
    for c in range(width):
        if all(grid[r][c] == ' ' for r in range(h)):
            sep.append(True)
        else:
            sep.append(False)

    # Find contiguous ranges of non-separator columns (each is a problem)
    problem_ranges = []
    in_problem = False
    start = 0
    for c in range(width):
        if not sep[c]:
            if not in_problem:
                in_problem = True
                start = c
        else:
            if in_problem:
                in_problem = False
                problem_ranges.append((start, c - 1))
    if in_problem:
        problem_ranges.append((start, width - 1))

    grand_total = 0

    for c_start, c_end in problem_ranges:
        # Find the operator row for THIS problem slice:
        # bottom-most row with + or * in [c_start, c_end]
        op_row = None
        for r in range(h):
            segment = grid[r][c_start:c_end + 1]
            if '+' in segment or '*' in segment:
                op_row = r

        if op_row is None:
            # No operator for this vertical slice -> skip
            continue

        # Determine the operator
        op_segment = grid[op_row][c_start:c_end + 1]
        if '+' in op_segment and '*' in op_segment:
            # If both appear (shouldn't in valid input), choose the leftmost
            plus_idx = op_segment.find('+') if '+' in op_segment else float('inf')
            mul_idx = op_segment.find('*') if '*' in op_segment else float('inf')
            op = add if plus_idx < mul_idx else mul
        elif '+' in op_segment:
            op = add
        elif '*' in op_segment:
            op = mul
        else:
            # No operator actually present
            continue

        # Build numbers column-by-column (each column is one number),
        # reading digits from top to just above op_row
        numbers = []
        for c in range(c_start, c_end + 1):
            digits = []
            for r in range(op_row):
                ch = grid[r][c]
                if ch.isdigit():
                    digits.append(ch)
            if digits:
                num = int(''.join(digits))
                numbers.append(num)

        if not numbers:
            continue

        result = reduce(op, numbers)
        grand_total += result

    return grand_total

*The program is correct:*

In [29]:
answer(6.2, 11159825706149, lambda:
       solve(lines))

Puzzle  6.2:   .0064 seconds, answer 11159825706149  correct

# [Day 7](https://adventofcode.com/2025/day/7): Gemini 3 Pro

*Gemini's code for **Part 1** is a bit verbose, but gets the job done.*

In [30]:
def solve_tachyon_manifold(puzzle_input):
    # 1. Parse the Grid
    grid = puzzle_input.splitlines()
    grid = [line for line in grid if line] # Remove empty lines
    
    if not grid:
        return 0

    height = len(grid)
    width = max(len(row) for row in grid)
    
    # Pad rows to ensure uniform width
    grid = [row.ljust(width, '.') for row in grid]

    # 2. Find Start Position 'S'
    # We scan the first row (or all rows if S is buried, but usually it's top)
    active_beams = set()
    for y in range(height):
        if 'S' in grid[y]:
            active_beams.add(grid[y].index('S'))
            # We assume beams start flowing from this row downwards
            # so we break and start the simulation from this row
            start_row = y
            break
    else:
        return 0 # No start found

    split_count = 0

    # 3. Simulate Row by Row
    # We iterate through the rows starting from the one containing 'S'
    for y in range(start_row, height):
        next_beams = set()
        
        # Check every column that currently has a beam
        for x in active_beams:
            cell = grid[y][x]
            
            if cell == '^':
                # SPLITTER: 
                # 1. Increment total split count
                split_count += 1
                
                # 2. Create new beams left and right
                # Check boundaries to ensure we don't wrap around or error
                if x - 1 >= 0:
                    next_beams.add(x - 1)
                if x + 1 < width:
                    next_beams.add(x + 1)
                    
            else:
                # EMPTY SPACE (.) or START (S):
                # The beam continues straight down
                next_beams.add(x)
        
        # Update the active beams for the next row iteration
        active_beams = next_beams
        
        # Optimization: If no beams are left, stop early
        if not active_beams:
            break

    return split_count

*The solution is correct:*

In [31]:
puzzle_input = get_text(7)

answer(7.1, 1681, lambda:
       solve_tachyon_manifold(puzzle_input))

Puzzle  7.1:   .0009 seconds, answer 1681            correct

*For **Part 2**, Gemini gets the key ideas (although it is a bit old-fashioned to use a `defaultdict(int)` instead of a `Counter`).*

In [32]:
from collections import defaultdict

def solve_quantum_tachyon_manifold(puzzle_input):
    # 1. Parse the Grid
    grid = puzzle_input.splitlines()
    grid = [line for line in grid if line]
    
    if not grid:
        return 0

    height = len(grid)
    width = max(len(row) for row in grid)
    # Pad rows to ensure uniform width
    grid = [row.ljust(width, '.') for row in grid]

    # 2. Find Start Position 'S'
    # 'counts' maps column_index -> number_of_timelines_at_this_column
    counts = defaultdict(int)
    start_row = 0
    found = False
    
    for y in range(height):
        if 'S' in grid[y]:
            start_x = grid[y].index('S')
            counts[start_x] = 1 # We start with 1 timeline
            start_row = y
            found = True
            break
            
    if not found:
        return 0

    # 3. Simulate Row by Row
    # We process the grid from the start row down to the bottom.
    # Each iteration calculates where the particles end up in the *next* row.
    for y in range(start_row, height):
        next_counts = defaultdict(int)
        
        for x, num_timelines in counts.items():
            # Determine what is at this position
            # If the timeline has drifted out of bounds, treat it as empty space (.)
            if 0 <= x < width:
                char = grid[y][x]
            else:
                char = '.'

            if char == '^':
                # SPLITTER:
                # The timelines here split. The count moves to both left and right.
                # If 5 timelines hit this splitter, 5 go left AND 5 go right.
                next_counts[x - 1] += num_timelines
                next_counts[x + 1] += num_timelines
            else:
                # EMPTY SPACE (.) or START (S):
                # The timelines pass straight through.
                next_counts[x] += num_timelines
        
        # Move to the next row
        counts = next_counts

    # 4. Sum Total Timelines
    # After processing the final row, 'counts' holds the number of timelines 
    # exiting the bottom of the manifold at each column.
    return sum(counts.values())

*The answer is correct:*

In [33]:
answer(7.2, 422102272495018, lambda:
       solve_quantum_tachyon_manifold(puzzle_input))

Puzzle  7.2:   .0017 seconds, answer 422102272495018 correct

*My friend Barry Hayes suggested I ask Gemini whose code it preferred, its or mine, So I asked Gemini: "**How would you compare your code to this:"***

In [34]:
def count_timelines(manifold: List[str]) -> int:
    """How many possible paths are there to the final line of the manifold?"""
    start = manifold[0].index('S')
    beams = Counter({start: 1})
    for line in manifold:
        for b, n in list(beams.items()):
            if line[b] == '^':
                beams[b]     -= n
                beams[b - 1] += n
                beams[b + 1] += n
    return sum(beams.values())

*Here is Gemini's reply:*

This is a great comparison. Your code is much more concise and "Pythonic," using collections.Counter and in-place updates to reduce verbosity. It strips away a lot of the boilerplate  that my solution included for robustness.

- **Your Code**: Assumes 'S' is always on the first line (manifold[0]). If 'S' is buried in the middle of the grid, your code crashes or misses it.
- **My Code**: Scans the whole grid for 'S', which handles cases where the entry point isn't at the very top.
- **Your Code**: Relies on Python's robust dictionary handling to avoid index errors (if b-1 is negative, it just adds a key to the dict). This is clever, but effectively allows particles to wrap around or exist in "negative space" if not careful (though typically harmless for this specific puzzle input).

*It seems to me that the instructions make it clear that there must be an 'S' on the top line, although technically it isn't stated explicitly. (If there is no 'S' on the top line, my code will return 0, not crash.) Similarly, the puzzle implies that beams can't go outside the borders, but Gemini is right that if the input file did have a splitter in the first or last column then there could be a problem that I don't check for.*

*Next I asked Gemini: **Do you think your code could be improved with some more modularity/abstraction, such as having a separate function to get input?** It replied:*

Yes, absolutely. Breaking the code down makes it much easier to read, test, and debug. If the input format changes (e.g., reading from a file vs. a raw string) or the simulation rules change (e.g., different splitter types), you only have to modify one specific function.

Here is a refactored version that separates Input Parsing, State Initialization, and Simulation Logic.

In [35]:
from collections import Counter
from typing import List, Tuple

def parse_manifold(input_text: str) -> List[str]:
    """
    Cleans and converts raw input text into a grid of strings.
    Handles stripping empty lines and padding if necessary.
    """
    lines = [line for line in input_text.splitlines() if line]
    if not lines:
        return []
    
    # Optional: Pad lines to uniform width for safety
    width = max(len(line) for line in lines)
    return [line.ljust(width, '.') for line in lines]

def find_start_position(grid: List[str]) -> Tuple[int, int]:
    """
    Locates the 'S' character in the grid.
    Returns (row_index, col_index).
    """
    for r, row in enumerate(grid):
        if 'S' in row:
            return r, row.index('S')
    raise ValueError("Start position 'S' not found in manifold.")

def process_row(beams: Counter, row_string: str) -> None:
    """
    Updates the beam positions for a single row of the manifold.
    Modifies the 'beams' Counter in-place using the subtraction logic.
    """
    # Create a snapshot of items to iterate over, since we modify 'beams' in the loop
    current_states = list(beams.items())
    
    for col, count in current_states:
        # Boundary check ensures we don't access outside the string
        if col < 0 or col >= len(row_string):
            continue
            
        char = row_string[col]
        
        if char == '^':
            # Splitter logic: Remove current stack, add to left and right
            beams[col] -= count
            beams[col - 1] += count
            beams[col + 1] += count
        # Else: beam passes through (no change needed for '.')

def solve_quantum_manifold(input_text: str) -> int:
    """
    Main orchestrator function.
    """
    # 1. Get Input
    grid = parse_manifold(input_text)
    
    # 2. Initialize State
    try:
        start_row, start_col = find_start_position(grid)
    except ValueError:
        return 0

    beams = Counter({start_col: 1})

    # 3. Run Simulation
    # We slice the grid to start from the row containing 'S'
    for row in grid[start_row:]:
        process_row(beams, row)

    # 4. Output Result
    return sum(beams.values())

*I like this refactored version better. It produces the correct answer:*

In [36]:
answer(7.2, 422102272495018, lambda:
       solve_quantum_manifold(puzzle_input))

Puzzle  7.2:   .0024 seconds, answer 422102272495018 correct

# [Day 8](https://adventofcode.com/2025/day/8): Claude Opus 4.5

*Here's Claude's code for **Part 1**:*

In [37]:
"""
Solution for the Junction Box puzzle.

Connect the 1000 pairs of junction boxes which are closest together.
Then multiply together the sizes of the three largest circuits.
"""

import sys
from collections import defaultdict

def parse_input(text):
    """Parse the input into a list of (x, y, z) coordinates."""
    boxes = []
    for line in text.strip().split('\n'):
        if line.strip():
            x, y, z = map(int, line.strip().split(','))
            boxes.append((x, y, z))
    return boxes

def distance_squared(p1, p2):
    """Calculate squared Euclidean distance between two points."""
    return (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2

class UnionFind:
    """Union-Find data structure for tracking connected components."""
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
        self.size = [1] * n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False  # Already in same circuit
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        self.size[px] += self.size[py]
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        return True
    
    def get_component_sizes(self):
        """Get sizes of all connected components."""
        components = defaultdict(int)
        for i in range(len(self.parent)):
            root = self.find(i)
            components[root] = self.size[root]
        return list(components.values())

def solve(boxes, num_connections=1000):
    """Connect the num_connections closest pairs and return product of 3 largest circuit sizes."""
    n = len(boxes)
    
    # Calculate all pairwise distances
    distances = []
    for i in range(n):
        for j in range(i + 1, n):
            dist_sq = distance_squared(boxes[i], boxes[j])
            distances.append((dist_sq, i, j))
    
    # Sort by distance
    distances.sort()
    
    # Use Union-Find to track circuits
    uf = UnionFind(n)
    
    # Make the specified number of connections
    connections_made = 0
    for dist_sq, i, j in distances:
        if connections_made >= num_connections:
            break
        # Attempt to connect (even if already in same circuit, it counts as a connection attempt)
        uf.union(i, j)
        connections_made += 1
    
    # Get component sizes and find the three largest
    sizes = uf.get_component_sizes()
    sizes.sort(reverse=True)
    
    # Multiply the three largest
    result = 1
    for i in range(min(3, len(sizes))):
        result *= sizes[i]
    
    return result

*Here I parse the input and hand it to Claude's function, which computes the correct answer:*

In [38]:
boxes = parse(8, ints, show=0)

answer(8.1, 24360, lambda:
       solve(boxes, 1000))

Puzzle  8.1:   .8309 seconds, answer 24360           correct

*Here is Claude's **Part 2** code:*

In [39]:
"""
Solution for the Junction Box puzzle - Part 2.

Connect junction boxes until they're all in one circuit.
Find the last two junction boxes connected and multiply their X coordinates.
"""

import sys
from collections import defaultdict

def parse_input(text):
    """Parse the input into a list of (x, y, z) coordinates."""
    boxes = []
    for line in text.strip().split('\n'):
        if line.strip():
            x, y, z = map(int, line.strip().split(','))
            boxes.append((x, y, z))
    return boxes

def distance_squared(p1, p2):
    """Calculate squared Euclidean distance between two points."""
    return (p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2

class UnionFind:
    """Union-Find data structure for tracking connected components."""
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n
        self.size = [1] * n
        self.num_components = n
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        px, py = self.find(x), self.find(y)
        if px == py:
            return False  # Already in same circuit
        if self.rank[px] < self.rank[py]:
            px, py = py, px
        self.parent[py] = px
        self.size[px] += self.size[py]
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        self.num_components -= 1
        return True

def solve_part2(boxes):
    """Connect boxes until all in one circuit, return product of X coords of last connection."""
    n = len(boxes)
    
    # Calculate all pairwise distances
    distances = []
    for i in range(n):
        for j in range(i + 1, n):
            dist_sq = distance_squared(boxes[i], boxes[j])
            distances.append((dist_sq, i, j))
    
    # Sort by distance
    distances.sort()
    
    # Use Union-Find to track circuits
    uf = UnionFind(n)
    
    # Keep connecting until all in one circuit
    last_i, last_j = None, None
    for dist_sq, i, j in distances:
        if uf.union(i, j):
            # This connection actually merged two circuits
            last_i, last_j = i, j
            if uf.num_components == 1:
                # All boxes are now in one circuit!
                break
    
    # Return product of X coordinates
    x1 = boxes[last_i][0]
    x2 = boxes[last_j][0]
    print(f"Last connection: {boxes[last_i]} and {boxes[last_j]}")
    print(f"X coordinates: {x1} and {x2}")
    return x1 * x2

*This code is correct. It does some extraneous printing, which is unneccessary, but not strictly incorrect.*

In [40]:
answer(8.2, 2185817796, lambda:
       solve_part2(boxes))

Last connection: (53148, 96207, 9295) and (41127, 88410, 10248)
X coordinates: 53148 and 41127


Puzzle  8.2:   .8300 seconds, answer 2185817796      correct

# [Day 9](https://adventofcode.com/2025/day/9): ChatGPT 5.1 Auto

*For **Part 1**, ChatGPT wrote a `main` program that reads from input and prints the answer. I told it **Refactor to have a function that takes the points as input and returns the area** and got this:*

In [41]:
def largest_rectangle_area(points):
    """
    Given a list of (x, y) points representing red tiles,
    return the largest possible rectangle area using any
    two points as opposite corners.
    """
    max_area = 0
    n = len(points)

    for i in range(n):
        x1, y1 = points[i]
        for j in range(i + 1, n):
            x2, y2 = points[j]

            width = abs(x2 - x1) + 1
            height = abs(y2 - y1) + 1
            area = width * height

            if area > max_area:
                max_area = area

    return max_area


In [42]:
points = parse(9, ints, show=0)

answer(9.1, 4772103936, lambda:
       largest_rectangle_area(points))

Puzzle  9.1:   .0282 seconds, answer 4772103936      correct

*For **Part 2**, ChatGPT considered a [scanline algorithm](https://en.wikipedia.org/wiki/Scanline_rendering), and then went for a [flood fill](https://en.wikipedia.org/wiki/Flood_fill). It assumed "I'm working with a grid that can have dimensions up to 2k x 2k, or 4 million cells, which seems manageable." So I told it **The grid can have dimensions up to 100,000 x 100,000 and there are about 500 red tile points**. ChatGPT came up with a [grid compression](https://stackoverflow.com/questions/29528934/coordinate-compression) method: define a grid where each red tile is in a separate cell, where the x and y extent of that cell spans an integer range (which must be recorded elsewhere). Now the grid is only of size roughly 500<sup>2</sup> instead of 100,000<sup>2</sup>.

In [43]:
from typing import List, Tuple, Dict

Point = Tuple[int, int]
Edge = Tuple[int, int, int, int]


def build_edges(points: List[Point]) -> List[Edge]:
    """
    Build polygon edges from the red points in loop order.
    Each edge is (x1, y1, x2, y2) between consecutive points (wrapped).
    """
    n = len(points)
    edges: List[Edge] = []
    for i in range(n):
        x1, y1 = points[i]
        x2, y2 = points[(i + 1) % n]
        edges.append((x1, y1, x2, y2))
    return edges


def point_in_or_on_polygon(x: int, y: int, edges: List[Edge]) -> bool:
    """
    Return True if lattice point (x, y) is inside the polygon or on its boundary.
    Polygon is rectilinear (axis-aligned), edges given by build_edges.
    Uses a ray-casting test along +x, counting intersections with vertical edges.
    """
    inside = False

    for (x1, y1, x2, y2) in edges:
        # Boundary check: point lies on this edge
        if x1 == x2:  # vertical edge
            if x == x1 and min(y1, y2) <= y <= max(y1, y2):
                return True
            # For ray cast, only vertical edges can intersect ray to +x
            # Use y-interval [min, max) to avoid double-counting vertices
            if (y1 > y) != (y2 > y) and x1 > x:
                inside = not inside
        else:  # horizontal edge
            if y == y1 and min(x1, x2) <= x <= max(x1, x2):
                return True
            # horizontal edges do not contribute to this vertical ray count

    return inside


def rect_crosses_polygon(
    xmin: int, xmax: int, ymin: int, ymax: int, edges: List[Edge]
) -> bool:
    """
    For a non-degenerate rectangle [xmin,xmax] x [ymin,ymax] (width>1,height>1),
    check if any polygon edge passes through the *interior* of that rectangle.
    Colinear edges on the boundary are allowed; only crossings inside are bad.
    """
    for (x1, y1, x2, y2) in edges:
        if x1 == x2:
            # vertical edge at x = x1
            x0 = x1
            if xmin < x0 < xmax:
                lo = max(min(y1, y2), ymin)
                hi = min(max(y1, y2), ymax)
                # If overlap has positive length, edge crosses interior
                if hi > lo:
                    return True
        else:
            # horizontal edge at y = y1
            y0 = y1
            if ymin < y0 < ymax:
                lo = max(min(x1, x2), xmin)
                hi = min(max(x1, x2), xmax)
                if hi > lo:
                    return True
    return False


def largest_rectangle_area_red_green(points: List[Point]) -> int:
    """
    Given the red tiles as vertices of an axis-aligned loop (in order),
    return the largest rectangle area such that:

    - Two opposite corners are red tiles (from `points`).
    - Every tile inside the rectangle is red or green, i.e., inside/on the loop.

    This works without building the full grid; it uses polygon geometry instead.
    """
    n = len(points)
    if n < 2:
        return 0

    edges = build_edges(points)

    # Cache point-in-polygon results so each lattice point is tested at most once
    point_cache: Dict[Point, bool] = {}

    def is_inside(x: int, y: int) -> bool:
        key = (x, y)
        if key in point_cache:
            return point_cache[key]
        val = point_in_or_on_polygon(x, y, edges)
        point_cache[key] = val
        return val

    max_area = 0

    for i in range(n):
        x1, y1 = points[i]
        for j in range(i + 1, n):
            x2, y2 = points[j]

            xmin, xmax = (x1, x2) if x1 <= x2 else (x2, x1)
            ymin, ymax = (y1, y2) if y1 <= y2 else (y2, y1)

            width = xmax - xmin + 1   # tiles
            height = ymax - ymin + 1  # tiles
            area = width * height

            if area <= max_area:
                continue

            # Thin rectangles (one row or one column): treat as 1D segments.
            if width == 1 or height == 1:
                if width == 1:
                    # Vertical segment at x = xmin from ymin..ymax
                    x = xmin
                    ok = True
                    for yy in range(ymin, ymax + 1):
                        if not is_inside(x, yy):
                            ok = False
                            break
                else:
                    # Horizontal segment at y = ymin from xmin..xmax
                    y = ymin
                    ok = True
                    for xx in range(xmin, xmax + 1):
                        if not is_inside(xx, y):
                            ok = False
                            break

                if ok:
                    max_area = area
                continue

            # Proper 2D rectangle: width>1 and height>1

            # The two given red corners are on the boundary by definition.
            # The other two corners must be inside/on polygon as well.
            if not is_inside(x1, y2):
                continue
            if not is_inside(x2, y1):
                continue

            # Ensure polygon boundary does not cut through the interior
            if rect_crosses_polygon(xmin, xmax, ymin, ymax, edges):
                continue

            max_area = area

    return max_area

In [44]:
answer(9.2, 1529675217, lambda:
       largest_rectangle_area_red_green(points))

Puzzle  9.2:   .7622 seconds, answer 1529675217      correct

# Summary

*Overall, the models did well, producing code that gives the correct answer in a reasonable run time. Some of the code could be improved stylistically.  (But I guess if you're vibe coding and not looking at the code, maybe that doesn't matter.) Here are the answers and the run times:*

In [45]:
summary(answers)

Puzzle  1.1:   .0012 seconds, answer 1182            correct
Puzzle  1.2:   .0017 seconds, answer 6907            correct
Puzzle  2.1:   .1217 seconds, answer 23560874270     correct
Puzzle  2.2:   .1282 seconds, answer 44143124633     correct
Puzzle  3.1:   .0038 seconds, answer 17085           correct
Puzzle  3.2:   .0069 seconds, answer 169408143086082 correct
Puzzle  4.1:   .0170 seconds, answer 1569            correct
Puzzle  4.2:   .0874 seconds, answer 9280            correct
Puzzle  5.1:   .0053 seconds, answer 635             correct
Puzzle  5.2:   .0002 seconds, answer 369761800782619 correct
Puzzle  6.1:   .0079 seconds, answer 5877594983578   correct
Puzzle  6.2:   .0064 seconds, answer 11159825706149  correct
Puzzle  7.1:   .0009 seconds, answer 1681            correct
Puzzle  7.2:   .0024 seconds, answer 422102272495018 correct
Puzzle  8.1:   .8309 seconds, answer 24360           correct
Puzzle  8.2:   .8300 seconds, answer 2185817796      correct
Puzzle  9.1:   .0282 sec